In [52]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.models import alexnet
import matplotlib.pyplot as plt
from torchvision.models import alexnet, AlexNet_Weights
import pandas as pd

In [53]:
# 学习率和批量大小组合
learning_rates = [0.001, 0.0001, 0.00005]
batch_sizes = [16, 64, 128] 
num_epochs = 50

In [54]:
# 保存结果的字典
results = {batch_size: [] for batch_size in batch_sizes}
data_to_save = []
data_each_epoch = []

In [55]:
# 定义图像转换
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # AlexNet uses 227x227 inputs, but 224x224 is also commonly used
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 定义数据集
class FlowerDataset(datasets.ImageFolder):
    def __init__(self, root_dir, transform=None):
        super().__init__(root=root_dir, transform=transform)

# 训练、测试和验证集加载器函数
def get_data_loaders(batch_size):
    train_dataset = FlowerDataset(root_dir='dataset/flowers/train', transform=transform)
    test_dataset = FlowerDataset(root_dir='dataset/flowers/test', transform=transform)
    val_dataset = FlowerDataset(root_dir='dataset/flowers/val', transform=transform)
    
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
    val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, test_loader, val_loader

# 函数计算准确度
def calculate_accuracy(model, loader):
    model.eval()  # Make sure model is in evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# 模型训练和评估函数
def train_and_evaluate(learning_rate, batch_size):
    train_loader, test_loader, val_loader = get_data_loaders(batch_size)
    
    model = alexnet(pretrained=False)
    model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 17)
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    performance_epoch = []
    
    # 训练模型
    model.train()  # Set model to training mode
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        test_accuracy = calculate_accuracy(model, test_loader)
        val_accuracy = calculate_accuracy(model, val_loader)
        performance_epoch.append([epoch, test_accuracy, val_accuracy])
    # 在测试集上评估
    
    
    # 在验证集上评估
    
    
    return test_accuracy, val_accuracy, performance_epoch





# Todo：在train and evaluate中加入 对每一个epoch都计算一遍accuracy的代码，并创建一个local list，在运行的时候塞进每一种情况中
#       直接用存储文件的csv画图
#       option1:上边的那行中只留下test，--> 在test中运行n个epoch，最后再跑一遍，使用validation
#       option2:每个都跑，跟上面一样




In [56]:
# 运行不同的学习率和批量大小组合，并记录结果
for batch_size in batch_sizes:
    for lr in learning_rates:
        test_acc, val_acc, performance_epoch = train_and_evaluate(lr, batch_size) # herehere
        results[batch_size].append(val_acc)
        for i in range(len(performance_epoch)):
            data_each_epoch.append([lr, batch_size, i + 1, performance_epoch[i][1], performance_epoch[i][2]])
        data_to_save.append([lr, batch_size, num_epochs, val_acc])
        print(f'LR: {lr}, Batch Size: {batch_size}, Validation Accuracy: {val_acc}')
        print(performance_epoch)

# 将数据保存到CSV文件
df = pd.DataFrame(data_to_save, columns=['Learning Rate', 'Batch Size', 'Epochs', 'Validation Accuracy'])
df.to_csv('AlexNet_model_performance.csv', mode='w', header=True, index=False)
df2 = pd.DataFrame(data_each_epoch, columns = ['Learning Rate', 'Batch Size', 'Epoch', 'Test Accuracy', 'Validation Accuracy'])
df2.to_csv('AlexNet_model_performance_each_epoch.csv', mode='w', header=True, index=False)

LR: 0.001, Batch Size: 16, Validation Accuracy: 49.26470588235294
[[0, 14.705882352941176, 7.352941176470588], [1, 13.970588235294118, 19.11764705882353], [2, 37.5, 27.205882352941178], [3, 39.705882352941174, 30.147058823529413], [4, 39.705882352941174, 37.5], [5, 44.85294117647059, 46.3235294117647], [6, 43.38235294117647, 32.35294117647059], [7, 46.3235294117647, 41.1764705882353], [8, 48.529411764705884, 44.85294117647059], [9, 48.529411764705884, 43.38235294117647], [10, 50.0, 44.85294117647059], [11, 50.73529411764706, 51.470588235294116], [12, 50.73529411764706, 45.588235294117645], [13, 50.73529411764706, 49.26470588235294], [14, 51.470588235294116, 38.23529411764706], [15, 51.470588235294116, 52.94117647058823], [16, 48.529411764705884, 47.05882352941177], [17, 49.26470588235294, 44.11764705882353], [18, 47.794117647058826, 52.94117647058823], [19, 51.470588235294116, 47.794117647058826], [20, 47.794117647058826, 44.85294117647059], [21, 44.11764705882353, 47.794117647058826],

In [ ]:
'''
# 绘制图表
plt.figure(figsize=(10, 8))  # 设置图表大小
for batch_size in batch_sizes:
    accs = results[batch_size]
    plt.plot(learning_rates, accs, 'o-', label=f'Batch Size {batch_size}')
    for i, txt in enumerate(accs):
        plt.text(learning_rates[i], accs[i], f'{txt:.2f}', fontsize=8, ha='center', va='bottom')

plt.xscale('log')
plt.xlabel('Learning Rate')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy for AlexNet Models with Different Batch Sizes')
plt.legend()
plt.grid(True)
plt.savefig('validation_accuracy_with_text.png')
plt.show()
'''


In [32]:
print(test_acc)
print(val_acc)

9.558823529411764
17.647058823529413


In [51]:
plt.figure(figsize=(10, 8))
for batch_size, metrics in results.items():
    # Extract the learning rates and accuracies
    lrs = metrics['learning_rates']
    test_accs = metrics['test_accuracies']
    val_accs = metrics['val_accuracies']

    # Plot test accuracies
    plt.plot(lrs, test_accs, 'o-', label=f'Batch Size {batch_size} Test')
    # Plot validation accuracies
    plt.plot(lrs, val_accs, 's--', label=f'Batch Size {batch_size} Validation')

    # Optionally, add text labels for each point
    for lr, test_acc, val_acc in zip(lrs, test_accs, val_accs):
        plt.text(lr, test_acc, f'{test_acc:.2f}', fontsize=8, ha='center', va='bottom')
        plt.text(lr, val_acc, f'{val_acc:.2f}', fontsize=8, ha='center', va='bottom')

plt.xscale('log')
plt.xlabel('Learning Rate')
plt.ylabel('Accuracy')
plt.title('Accuracies for AlexNet Models with Different Batch Sizes and Learning Rates')
plt.legend()
plt.grid(True)
plt.show()

TypeError: list indices must be integers or slices, not str

<Figure size 1000x800 with 0 Axes>

In [26]:
print(results)
print(learning_rates)
print(test_accuracy)

{32: [11.764705882352942, 20.58823529411765], 64: [9.558823529411764, 16.176470588235293]}
[0.001, 0.0005]


NameError: name 'test_accuracy' is not defined

In [27]:
data_to_save

[[0.001, 32, 2, 11.764705882352942],
 [0.0005, 32, 2, 20.58823529411765],
 [0.001, 64, 2, 9.558823529411764],
 [0.0005, 64, 2, 16.176470588235293]]

In [28]:
output.shape

NameError: name 'output' is not defined